In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import axes3d  
import seaborn as sns
from scipy import stats

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn import svm
import random
from imblearn.over_sampling import SMOTE
from EDA_helper import *
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.feature_selection import RFE, RFECV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [2]:
ratings = pd.read_excel("../datasets/ratings_2_06-17.xlsx")
ratio = pd.read_excel("../datasets/ratio_figs_2.xlsx")
ratio = ratio.rename(columns = {'Public Date':'Data Date','EXCHANGE TICKER SYMBOL - HISTORICAL':'Ticker Symbol'})


ratio1, ratings1 = format_df(ratio.copy(), ratings.copy(), ['Trailing P/E to Growth (PEG) ratio','Dividend Yield'])
numer = list(ratio1.columns[4:-1])
X1,Y1,full1 = join_numerical(ratio1, ratings1, numerical=numer)

Unique Companies in Ratings:  88
Unique Companies in Features:  42
Intersection of companies:  26


In [11]:
scl_X = StandardScaler().fit_transform(X1)
enc_Y, _ = encode(Y1, form='full')
# enc_Y = Y1.T[0]

In [123]:
# comps = list(full1['Ticker Symbol'].unique())
# np.random.shuffle(comps)
# split = int(np.round(0.8*len(comps)))
# train_comps = comps[:split]
# test_comps = comps[split:]
# train_strat = []
# test_strat = []
# for comp in train_comps:
#     train_strat.extend(full1.index[full1['Ticker Symbol'] == comp].tolist())
# for comp in test_comps:
#     test_strat.extend(full1.index[full1['Ticker Symbol'] == comp].tolist())

# assert len(test_strat)/len(full1.index) + len(train_strat)/len(full1.index) == 1.0

In [126]:
# X_train = scl_X[train_strat,:]
# X_test = scl_X[test_strat, :]
# Y_train = enc_Y[train_strat]
# Y_test = enc_Y[test_strat]

In [127]:
# pd.Series(Y_test).value_counts()

In [98]:
# X_train, Y_train, X_val, Y_val, X_test, Y_test = data_split(scl_X, enc_Y, full1, split=[0.8, 0.0],smote=False, seed=4)

In [12]:
X_train, X_test,Y_train, Y_test = train_test_split(scl_X, enc_Y, test_size=0.2, random_state=100)

In [13]:

param_grid = [
  {'C': [0.001, 0.01, 0.1, 0.5, 1.0, 10.0, 100.0, 100.0, 2000.0], 'kernel': ['linear']},
  {'C': [0.001, 0.01, 0.1, 0.5, 1.0, 10.0, 100.0, 100.0,2000.0], 'gamma': [0.01, 0.001, 0.0001], 'kernel': ['rbf']},
    {'C': [0.001, 0.01, 0.1, 0.5, 1.0, 10.0, 100.0, 100.0,2000.0], 'degree': [2, 3, 4, 5, 6,7,8,9], 'kernel': ['poly']},
 ]
strat = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)

svc = svm.SVC()
svm_cv = GridSearchCV(svc, param_grid, scoring='accuracy')
svm_cv_strat = GridSearchCV(svc, param_grid, scoring='accuracy', cv=strat)

svm_cv.fit(X_train, Y_train)
svm_cv_strat.fit(X_train, Y_train)

c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), U

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=3, random_state=1),
             estimator=SVC(),
             param_grid=[{'C': [0.001, 0.01, 0.1, 0.5, 1.0, 10.0, 100.0, 100.0,
                                2000.0],
                          'kernel': ['linear']},
                         {'C': [0.001, 0.01, 0.1, 0.5, 1.0, 10.0, 100.0, 100.0,
                                2000.0],
                          'gamma': [0.01, 0.001, 0.0001], 'kernel': ['rbf']},
                         {'C': [0.001, 0.01, 0.1, 0.5, 1.0, 10.0, 100.0, 100.0,
                                2000.0],
                          'degree': [2, 3, 4, 5, 6, 7, 8, 9],
                          'kernel': ['poly']}],
             scoring='accuracy')

In [14]:
print(svm_cv.best_params_,svm_cv.best_score_)
print(svm_cv_strat.best_params_, svm_cv_strat.best_score_)

{'C': 2000.0, 'gamma': 0.01, 'kernel': 'rbf'} 0.7970149253731342
{'C': 2000.0, 'gamma': 0.01, 'kernel': 'rbf'} 0.7955081322514058


In [15]:
cv_preds = svm_cv.predict(X_test)
np.mean(cv_preds == Y_test)
print(classification_report(Y_test, cv_preds))
cv_preds1 = svm_cv_strat.predict(X_test)
np.mean(cv_preds1 == Y_test)
print(classification_report(Y_test, cv_preds1))

              precision    recall  f1-score   support

          -6       0.00      0.00      0.00         0
          -4       0.33      0.50      0.40         6
          -3       0.73      0.73      0.73        11
          -2       0.69      0.45      0.55        20
          -1       0.64      0.76      0.70        21
           0       0.84      0.86      0.85        56
           1       0.87      0.83      0.85        24
           2       0.92      0.85      0.88        13
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         9
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         5

    accuracy                           0.79       168
   macro avg       0.75      0.75      0.75       168
weighted avg       0.80      0.79      0.79       168

              precision    recall  f1-score   support

          -6       0.00      0.00      0.00         0
          -4       0.33 

c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
svc_rfe = svm.SVC(kernel='linear', C=10.0)
rfe = RFECV(estimator=svc_rfe, min_features_to_select=2, step=1)

# rfe1 = RFECV(estimator=qda, n_features_to_select=2, step=1)
rfe.fit(X_train, Y_train)
# rfe1.fit(X1, Y1)
sups = rfe.support_
# sups1 = rfe1.support_
X2 = X_train[:, sups]
Y2 = Y1.copy
# Y3, full3
# X2


In [ ]:
r_preds = rfe.predict(X_val)
np.mean(r_preds == Y_val)
classification_report(Y_val, )

In [ ]:
svvvvc = svm.SVC(kernel='linear', C=10.0)
svvvvc.fit(X2, Y1)
s_preds = svvvvc.predict(X2)
np.mean(s_preds == Y1)



In [11]:
pd.Series(enc_Y).value_counts()

3    496
2    196
5     47
4     43
6     41
1     15
dtype: int64

In [7]:
C = 10.0  # SVM regularization parameter

svc = svm.SVC(kernel='linear', C=C).fit(X1, )
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C).fit(tr_x, tr_y)
poly_svc = svm.SVC(kernel='poly', degree=3, C=C).fit(tr_x, tr_y)
lin_svc = svm.LinearSVC(C=C).fit(tr_x, tr_y)

NameError: name 'tr_x' is not defined

In [ ]:
# preds = svc.predict(two_x)
# preds1 = rbf_svc.predict(two_x)
# preds2 = poly_svc.predict(two_x)
# preds3 = lin_svc.predict(two_x)


preds = svc.predict(cca_tex)
preds1 = rbf_svc.predict(cca_tex)
preds2 = poly_svc.predict(cca_tex)
preds3 = lin_svc.predict(cca_tex)
print("linear kernel :",'\n', pd.Series(preds).value_counts(),'\n',
"RBF kernel :",'\n',pd.Series(preds1).value_counts(),'\n',
"Poly kernel :",'\n',pd.Series(preds2).value_counts(),'\n',
"linear SVC :",'\n',pd.Series(preds3).value_counts(),'\n',
"True dist. :",'\n',pd.Series(te_y).value_counts())

In [ ]:
titles = ['SVC with linear kernel',
          'LinearSVC (linear kernel)',
          'SVC with RBF kernel',
          'SVC with polynomial (degree 3) kernel']

# low','med', 'high'
for i, clf in enumerate((svc, lin_svc, rbf_svc, poly_svc)):
    print(titles[i], '\n')
    print(classification_report(te_y, pd.Series(clf.predict(cca_tex)).apply(lambda x: int(x)), target_names=['B','BB','BBB','A','AA','AAA']))

In [2]:
def score_one_off(clf, preds, true):
    """ 
    preds/true: must be array of int
    """
    assert len(preds) == len(true)
    bools = np.zeros(len(preds))
    for i in range(len(preds)):
        bools[i] = (preds[i] == true[i]) | (preds[i] == (true[i] + 1)) | (preds[i] == (true[i] - 1)) 
    return np.mean(bools)